In [4]:
import pandas as pd
import plotly_express as px

In [5]:
athlete_events = pd.read_csv("../Databehandling-Projekt/data/athlete_events.csv")
Noc_regions = pd.read_csv("../Databehandling-Projekt/data/noc_regions.csv")

df_athlete = pd.DataFrame(athlete_events)
df_regions = pd.DataFrame(Noc_regions)

FileNotFoundError: [Errno 2] No such file or directory: '../Databehandling-Projekt/data/athlete_events.csv'

## Uppgift 1 - Landstatistik
---
Landet vi blivit tilldelade är Storbrittanien, i datan representerat som "NOC": "GBR".

Till att börja med ska kolumnen över idrottarnas namn anonymiseras m.h.a hashfunktionen SHA-256. Vi gör detta men kommer behålla namnen i datan tills vidare för att kunna göra några roliga diagram över medaljstatistik som blir mer intressanta när namnen är med.

Vidare ska följande visualiseras:
- de sporter landet fått flest medaljer i
- antal medaljer per OS
- histogram över åldrar

In [ ]:
# kopierar "Namn" kolumnen och lägger till en ny kolumn "Hashed-name" som vi kan utföra Hashen på.

df_athlete["Hashed-name"] = df_athlete["Name"].copy()
df_athlete.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Hashed-name
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,A Dijiang
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,A Lamusi
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,Gunnar Nielsen Aaby
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,Edgar Lindenau Aabye
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN,Christine Jacoba Aaftink


In [ ]:
df_UK = df_athlete[df_athlete['NOC'].isin(['GBR'])]
df_UK.head()

NameError: name 'df_athlete' is not defined

In [ ]:
import hashlib as hl

df_athlete["Hashed-name"] = df_athlete["Hashed-name"].apply(
    lambda x:
        hl.sha256(x.encode()).hexdigest()
)
df_athlete.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Hashed-name
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,d392118055bf5b1d3aa9cc021a4b5cdb0a5cfa372aeb09...
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,a8835de22218cfbffb0aedd295b8f0d1d25d2baac11916...
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,4eb420cc4e630192b6b51d2578d94c53dc136364cd5ac0...
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,7548f189f592b624ac33c41bcd75354c55d54b648c1ab9...
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN,fe7aa4faa328e7be665ae4be9e9b3222e667c5e276c38d...


In [ ]:
# Sparar ner detta till en ny fil i data-mappen

df_athlete.to_csv("../Databehandling-Projekt/data/athlete_events_semi-anonymized.csv")

In [ ]:
medal_UK = df_UK[['Sport', 'Medal']].groupby("Sport").count().reset_index().sort_values(by="Medal", ascending=False)

fig_medals = px.bar(medal_UK,
                    x='Sport',
                    y='Medal',
                    )
fig_medals.show()


---

### Antal medaljer per OS <br>

---

In [ ]:
# Börjar med att filtrera ut Storbrittaninen ur datan
UK_athletes = athlete_events[athlete_events['NOC'] == 'GBR']

# Räknar alla medaljer vunna i OS och tar sedan bort alla rader utan NaN värden.
UK_seasonal = UK_athletes[UK_athletes['Medal'].notna()].groupby(['Year', 'Season'])['Medal'].count().reset_index()

# Byter namn på Medal kolumnen för tydlighet
UK_seasonal.rename(columns={'Medal': 'Total Medals'}, inplace=True)

UK_seasonal.head()

,Year,Season,Total Medals
0,1896,Summer,9
1,1900,Summer,108
2,1904,Summer,2
3,1906,Summer,39
4,1908,Summer,368


In [ ]:

fig = px.bar(UK_seasonal,
             x='Year', y='Total Medals', color='Season',
             title='Total Olympic Medals Won by Great Britain by Year and Season',
             labels={'Year': 'Olympic Year', 'Total Medals': 'Total medals won'},
             )

fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()



# UK_seasonal.head()

## Histogram över åldrar
---

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# Plockar ut 4 set som jag försökt göra self-explanatory. Men jag hoppas att det ska ge nånting som är visuellt intressant.. 
# Till att börja med kollar vi åldrarna men funkar detta bra så kan vi kanske göra en gender-plot också.
# Eventuellt en sån där click-wheel där man kan välja tidsspann och få upp en tydlig gender-bild, exempelvis.

pre_ww1 = UK_athletes[UK_athletes['Year'] < 1940]

between_wars = UK_athletes[(UK_athletes['Year'] >= 1914) & (UK_athletes['Year'] < 1945)]

post_ww2_pre_2000 = UK_athletes[(UK_athletes['Year'] >= 1945) & (UK_athletes['Year'] < 2000)]

post_2000 = UK_athletes[UK_athletes['Year'] >= 2000]


# Nu plotts

fig_pre_ww1 = px.histogram(pre_ww1, x='Age', nbins=50,
                           title="Age Distribution of UK's Athletes Before WWI",
                           labels={'Age': 'Age of Athletes'})

fig_between_wars = px.histogram(between_wars, x='Age', nbins=50,
                           title="Age Distribution of UK's Athletes between WWI & WWII",
                           labels={'Age': 'Age of Athletes'})

fig_post_ww2 = px.histogram(post_ww2_pre_2000, x='Age', nbins=50,
                           title="Age Distribution of UK's Athletes After WWII",
                           labels={'Age': 'Age of Athletes'})

fig_post_2000 = px.histogram(post_2000, x='Age', nbins=50,
                           title="Age Distribution of UK's Athletes After 2000s",
                           labels={'Age': 'Age of Athletes'})

#fig_pre_ww1.show()
#fig_between_wars.show()
#fig_post_ww2.show()
#fig_post_2000.show()

# Creating a subplot with 4 rows to display all histograms together
fig_all = make_subplots(rows=4, cols=1, subplot_titles=(
    "Age Distribution of UK's Athletes Before WWI",
    "Age Distribution of Britains Athletes between WWI & WWII",
    "Age Distribution of Britains Athletes After WWII",
    "Age Distribution of Britains Athletes After 2000s"
))

# Adding each histogram to the subplot
fig_all.add_trace(go.Histogram(x=pre_ww1['Age'], nbinsx=50), row=1, col=1)
fig_all.add_trace(go.Histogram(x=between_wars['Age'], nbinsx=50), row=2, col=1)
fig_all.add_trace(go.Histogram(x=post_ww2_pre_2000['Age'], nbinsx=50), row=3, col=1)
fig_all.add_trace(go.Histogram(x=post_2000['Age'], nbinsx=50), row=4, col=1)

# Update layout
fig_all.update_layout(height=800, width=700, title_text="Age Distribution of UK's Athletes Across Different Eras")
fig_all.update_xaxes(title_text="Age", row=4, col=1)
fig_all.update_yaxes(title_text="Number of Athletes", col=1)

# Show plot
fig_all.show()

In [ ]:
UK_athletes.head()
df_age = UK_athletes[['Age', 'Name']].groupby(['Name', 'Age']).count().reset_index()
df_age.head()

# Medelålder i datan
df_age['Age'].mean()



27.010295040803516

In [ ]:
fig = px.histogram(df_age, x='Age', title="Age distribution UK's Data")

fig.show()

In [ ]:
# Medelålder Kvinnor i datan
wom = ['F']
df_age = UK_athletes[UK_athletes["Sex"].isin(wom)]

fig = px.histogram(df_age, x='Age', title='Women age distribution')
fig.show()

In [ ]:
men = ['M']
df_age = UK_athletes[UK_athletes['Sex'].isin(men)]

fig = px.histogram(df_age, x='Age', title='Men age distribution')
fig.show()

In [ ]:
# Medelålder av medaljörer
df_mean_age = UK_athletes[['Age', 'Medal']].groupby('Medal').mean().reset_index().sort_index().sort_values(by='Medal')
display(df_mean_age.head())

fig = px.bar(df_mean_age, x='Medal', y='Age', title="Mean Age of UK's medal-holders")
fig.show()

,Medal,Age
0,Bronze,26.556090
1,Gold,28.951294
2,Silver,27.961988


In [ ]:
# Yngsta medaljörer

df_age_min = UK_athletes[['Age', 'Medal']].groupby('Medal').min().reset_index().sort_index().sort_values(by='Medal')
df_age_min.head()

,Medal,Age
0,Bronze,15.0
1,Gold,17.0
2,Silver,15.0


In [ ]:
# Äldsta medaljörer

df_age_max = UK_athletes[['Age', 'Medal']].groupby('Medal').max().reset_index().sort_index().sort_values(by='Medal')
df_age_max

,Medal,Age
0,Bronze,57.0
1,Gold,60.0
2,Silver,73.0
